In [1]:
# Importing all required libraries
# !pip install bs4
# !pip install matplotlib
# !pip install seaborn
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
from bs4 import BeautifulSoup
from numpy import median

In [2]:
url = "https://www.google.com/travel/hotels/Mumbai?g2lb=2502548%2C4258168%2C4270442%2C4306835%2C4317915%2C4371335%2C4401769%2C4419364%2C4429192%2C4482438%2C4486153%2C4509341%2C4524134%2C4533547%2C4533548%2C4533882%2C4536454%2C4547627%2C4270859%2C4284970%2C4291517&hl=en-IN&gl=in&cs=1&ssta=1&ap=aAE&q=juhu%20hotels&rp=EMaW74Pl8vaAChCmiL67me393GwQqqT4zp6RhsMSEPOBovqAg7-zBzgBQABIAqIBGUp1aHUsIE11bWJhaSwgTWFoYXJhc2h0cmE&ictx=1&sa=X&ved=0CAAQ5JsGahcKEwjYj-Oj4uHwAhUAAAAAHQAAAAAQBw&utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAESABo9Ch8SGzoZSnVodSwgTXVtYmFpLCBNYWhhcmFzaHRyYRoAEhoSFAoHCOUPEAUYGRIHCOUPEAUYGhgBMgIQACoPCgsoAUoCIAE6A0lOUhoA"  # "https://www.travelguru.com/hotels/5-star-hotels-near-juhu-beach-in-mumbai"

r = requests.get(url)
htmlContent = r.content

soup = BeautifulSoup(htmlContent, 'html.parser')

In [43]:
def get_data(): 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get(url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    alls = []
    for d in soup.findAll('div', attrs={'class':"yGKfaf ev5GFe"}):
        hotel = d.find('h2', attrs={'class':"BgYkof ogfYpf ykx2he"})
        rating = d.find('span', attrs={'class':"KFi5wf lA0BZ"})
        customers_rated = d.find('span', attrs={'class':"jdzyld XLC8M"})
        price = d.find('span', attrs={'class':"W9vOvb nDkDDb"})
        all1=[]
        if hotel is not None:
            all1.append(hotel.text)
        else:
            all1.append("unknown")
        if rating is not None:
            all1.append(rating.text)
        else:    
            all1.append('0')
        if customers_rated is not None:
            all1.append(customers_rated.text)
        else:
            all1.append('0')     
        if price is not None:
            all1.append(price.text)
        elif price is None:
            try :
                price = d.find('div', attrs={'class':"JGa7fd"}).get('aria-label')
            except :
                pass  #  get('aria-label')
            if price is not None:
                all1.append(price)
            else:    
                all1.append(None)
        alls.append(all1)
    return alls
get_data()

[['Hotel Karl Residency', '3.8', ' (2,326)', '₹3,935'],
 ['OYO Flagship 18502 Sahar International', '3.7', ' (482)', '₹1,116'],
 ['Novotel Mumbai Juhu Beach', '4.3', ' (14,913)', '₹3,918'],
 ['Hotel Sea Princess', '4.3', ' (6,553)', '₹3,389'],
 ['JW Marriott Mumbai Juhu', '4.5', ' (21,726)', '₹6,161'],
 ['Hotel Milan International', '3.4', ' (717)', '₹1,534'],
 ['Ramee Guestline Hotel Juhu', '3.9', ' (2,010)', '₹2,308'],
 ['Sun-n-Sand Hotel, Juhu, Mumbai', '4.3', ' (4,700)', '₹3,889'],
 ['Citizen Hotel', '4.0', ' (1,213)', '₹2,123'],
 ['The Park Mumbai Juhu', '4.3', ' (549)', None],
 ['Hotel Sahara Star', '4.5', ' (33,962)', '₹3,864'],
 ['Hotel Karl Residency', '3.8', ' (2,326)', '₹842'],
 ['T24 Retro - 4 Star Boutique Hotel', '4.6', ' (200)', '₹1,886'],
 ['Ramada Plaza by Wyndham Palm Grove', '4.2', ' (8,415)', '₹3,696']]

In [44]:
results = []
results.append(get_data())
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Hotel', 'Rating', 'Customers_Rated', 'Price'])
# df.to_csv('hotel_data.csv', index=False, encoding='utf-8')
df

,Hotel,Rating,Customers_Rated,Price
0,Hotel Bawa Continental,4.1,"(1,545)","₹2,318"
1,ibis Mumbai Airport,4.1,"(5,482)","₹2,490"
2,Novotel Mumbai Juhu Beach,4.3,"(14,913)","₹3,918"
3,Hotel Sea Princess,4.3,"(6,553)","₹3,389"
4,JW Marriott Mumbai Juhu,4.5,"(21,726)","₹6,161"
5,Hotel Milan International,3.4,(717),"₹1,534"
6,Ramee Guestline Hotel Juhu,3.9,"(2,010)","₹2,308"
7,"Sun-n-Sand Hotel, Juhu, Mumbai",4.3,"(4,700)","₹3,889"
8,Citizen Hotel,4.0,"(1,213)","₹2,123"
9,The Park Mumbai Juhu,4.3,(549),None


In [45]:
df.dropna(inplace=True)
df.Price = df.Price.str.replace(r'\D+', '').astype('int')
# df.Price = df.Price.astype(int)
df.Rating = pd.to_numeric(df.Rating)
df.Customers_Rated = df.Customers_Rated.str.replace(r'\D+', '').astype('int')
# df.Customers_Rated = pd.to_numeric(df.Customers_Rated)
df = df.drop_duplicates(subset="Hotel")
# df = df.reset_index()
df = df.sort_values('Price', ascending=False)
df

<ipython-input-45-a6c159c91c57>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Price = df.Price.str.replace(r'\D+', '').astype('int')
<ipython-input-45-a6c159c91c57>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Customers_Rated = df.Customers_Rated.str.replace(r'\D+', '').astype('int')


,Hotel,Rating,Customers_Rated,Price
4,JW Marriott Mumbai Juhu,4.5,21726,6161
2,Novotel Mumbai Juhu Beach,4.3,14913,3918
7,"Sun-n-Sand Hotel, Juhu, Mumbai",4.3,4700,3889
10,Hotel Sahara Star,4.5,33962,3864
13,Ramada Plaza by Wyndham Palm Grove,4.2,8415,3696
3,Hotel Sea Princess,4.3,6553,3389
1,ibis Mumbai Airport,4.1,5482,2490
0,Hotel Bawa Continental,4.1,1545,2318
6,Ramee Guestline Hotel Juhu,3.9,2010,2308
8,Citizen Hotel,4.0,1213,2123


In [6]:
# # Plotting graph for price, rating and review comparison
# plt.figure(figsize=(20,20))
# sns.axes_style('white')
# sns.set_color_codes("muted")
# sns.set_context("notebook", font_scale=1.8, rc={"lines.linewidth": 1})
# # Plot -1
# plt.subplot(3,1,1)
# sns.despine(left=True, bottom=True)
# plt.title('Hotel Prices, Ratings and Total Customers Rated Comparison')
# plot1 = sns.barplot(x=df.Price, y=df.Hotel, data=df, alpha=.3, estimator=median, ci=None, palette="deep", saturation=2.5)
# plot1.tick_params(left=False,bottom=False)
# plot1.set(xticklabels=[])
# for i, v in enumerate(df.Price):
#     plt.text(v - 650, i + .25, u"\u20B9"+str(v), color = 'black')
# # Plot -2
# plt.subplot(3,1,2)
# sns.despine(left=True, bottom=True)
# plot2 = sns.barplot(x=df.Rating, y=df.Hotel, data=df, alpha=.5, estimator=median, ci=None, color='skyblue', saturation=2.5)
# plot2.tick_params(left=False,bottom=False)
# plot2.set(xticklabels=[])
# for i, v in enumerate(df.Rating):
#     plt.text(v - 0.23, i + .25, str(v)+u"\u2605", color = 'black')
# # Plot -3
# plt.subplot(3,1,3)
# sns.despine(left=True, bottom=True)
# plot3 = sns.barplot(x=df.Customers_Rated, y=df.Hotel, data=df, alpha=.3, estimator=median, ci=None, palette="deep", saturation=2.5)
# plot3.tick_params(left=False,bottom=False)
# plot3.set(xticklabels=[])
# for i, v in enumerate(df.Customers_Rated):
#     plt.text(v - 800, i + .25, str(v), color = 'black')

In [7]:
# test_str = "abbabba"
  
# # printing original string 
# print("The original string is : " + str(test_str))
  
# # Using nested replace()
# # Replace multiple characters at once
# res = test_str.replace('a', '%temp%').replace('b', 'a').replace('%temp%', 'b')
  
# # printing result 
# print("The string after replacement of positions : " + res)

In [8]:
# for d in soup.findAll('div', attrs={'class':"yGKfaf ev5GFe"}): 
#     try :
#         price = d.find('div', attrs={'class':"JGa7fd"}).get('aria-label')
#     except :
#         pass
#     print(price)

In [9]:
# text = soup.get_text()
# text

In [10]:
# tokens = [t for t in text.split(',')]
# tokens

In [11]:
# from nltk.corpus import stopwords

# sword = stopwords.words('english')

# list_of_words_ = []
# for word in tokens:
#     if word.lower() not in  sword:
#         list_of_words_.append(word.lower())

# list_of_words_

In [12]:
# anchors = soup.find_all('a')
# anchors

In [13]:
# string = str(anchors)
# string = string.split('><')
# string

In [14]:
# for word in string:
#     if word.startswith('/a>'):
#         sorted = word.split('=')
#         print(sorted)

In [15]:
# all_links = set()
# print(soup.find('a').get_text())
# for link in anchors:
#     if(link.get('href') != '#'):
#         linkText = link.get('href')
#         all_links.add(link)
#         print(linkText)

In [16]:
# Importing all required libraries
# import pandas as pd
# import requests
# import numpy as  np
# import re
# import nltk
# from bs4 import BeautifulSoup
    
# # URL of the site from where data is extracted using request and Beautifulsoup libraries
# url = "https://www.google.com/travel/hotels/Mumbai?g2lb=2502548%2C4258168%2C4270442%2C4306835%2C4317915%2C4371335%2C4401769%2C4419364%2C4429192%2C4482438%2C4486153%2C4509341%2C4524134%2C4533547%2C4533548%2C4533882%2C4536454%2C4547627%2C4270859%2C4284970%2C4291517&hl=en-IN&gl=in&cs=1&ssta=1&ap=aAE&q=juhu%20hotels&rp=EMaW74Pl8vaAChCmiL67me393GwQqqT4zp6RhsMSEPOBovqAg7-zBzgBQABIAqIBGUp1aHUsIE11bWJhaSwgTWFoYXJhc2h0cmE&ictx=1&sa=X&ved=0CAAQ5JsGahcKEwjYj-Oj4uHwAhUAAAAAHQAAAAAQBw&utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAESABo9Ch8SGzoZSnVodSwgTXVtYmFpLCBNYWhhcmFzaHRyYRoAEhoSFAoHCOUPEAUYGRIHCOUPEAUYGhgBMgIQACoPCgsoAUoCIAE6A0lOUhoA"  # "https://www.travelguru.com/hotels/5-star-hotels-near-juhu-beach-in-mumbai"

# r = requests.get(url)
# htmlContent = r.content

# soup = BeautifulSoup(htmlContent, 'html.parser')
    
# # From the html data specific class deta is been extracted usind find_all
# match = soup.find_all('a', class_="OxGZuc W8vlAc lRagtb")
# match = list(match)
    
# required data 
# a1 = str(match[0])
# print(a1[14:67])

# a2 = str(match[1])
# print(a2[14:64])

# a3 = str(match[2])
# print(a3[14:70])

# a4 = str(match[3])
# print(a4[14:63])

# a5 = str(match[4])
# print(a5[14:68])

# a6 = str(match[5])
# print(a6[14:70])

# a7 = str(match[6])
# print(a7[14:70])

# a8 = str(match[7])
# print(a8[14:65])

# a9 = str(match[8])
# print(a9[14:58])

# a10 = str(match[9])
# print(a10[14:71])

# a11 = str(match[10])
# print(a11[14:78])

# a12 = str(match[11])
# print(a12[14:75])

In [17]:
# layer1=match
# layer2=[]
# while True:
#     for i in layer1:
#         if isinstance(i,type):
#             for j in i:
#                 layer2.append(j)
#         else:
#             print(i)
#     layer1[:]=layer2
#     layer2=[]
#     if len(layer1)==0:
#         break

In [18]:
# match = soup.find_all('a', class_="OxGZuc W8vlAc lRagtb")
# match = list(match)

In [19]:
# match1 = soup.find_all('a', class_="spNMC lRagtb")
# match1 = list(match1)
# match1

In [20]:
# ABCD = []
# for a in match1:
#     pattern = """
#     (?P<Rating>[\d+].*)
#     (\ out\ of\ 5\ stars\ from\ )
#     (?P<Total_Reviews>.*)
#     (\ reviews\,\ )
#     (?P<Hotel>.*)
#     (\"\ class\=\"spNMC\ lRagtb\")
#     """
#     for item in re.finditer(pattern, str(a), re.VERBOSE):
#         ABCD.append(item.groupdict())
# ABCD    

In [21]:
# Handle the Clipboard Content and Launch the Browser
# mapIt.py - Launches a map in the browser using an address from the
# command line or clipboard.
# !pip install pyperclip
# import webbrowser, sys, pyperclip
# if len(sys.argv) > 1:    
#     # Get address from command line.    
#     address = ' '.join(sys.argv[1:])
# else:    
#     # Get address from clipboard.    
#     address = pyperclip.paste()
# webbrowser.open('https://www.google.com/maps/place/' + address)

In [22]:
# import requests, sys, webbrowser, bs4
# print('Googling...')    # display text while downloading the Google page
# res = requests.get('http://google.com/search?q=' + ' '.join(sys.argv[1:]))
# res.raise_for_status()

In [23]:
# !pip install selenium
# !pip install webdriver-manager
# from selenium import webdriver

# browser = webdriver.Firefox()
# type(browser)
# browser.get('http://inventwithpython.com')

In [24]:
# def hotels_data():
#     # Importing all required libraries
#     #     !pip install bs4
#     import pandas as pd
#     import requests
#     import numpy as  np
#     import re
#     from bs4 import BeautifulSoup
#     # URL of the site from where data is extracted using request and BeautifulSoup libraries
#     url = "https://www.google.com/travel/hotels/Mumbai?g2lb=2502548%2C4258168%2C4270442%2C4306835%2C4317915%2C4371335%2C4401769%2C4419364%2C4429192%2C4482438%2C4486153%2C4509341%2C4524134%2C4533547%2C4533548%2C4533882%2C4536454%2C4547627%2C4270859%2C4284970%2C4291517&hl=en-IN&gl=in&cs=1&ssta=1&ap=aAE&q=juhu%20hotels&rp=EMaW74Pl8vaAChCmiL67me393GwQqqT4zp6RhsMSEPOBovqAg7-zBzgBQABIAqIBGUp1aHUsIE11bWJhaSwgTWFoYXJhc2h0cmE&ictx=1&sa=X&ved=0CAAQ5JsGahcKEwjYj-Oj4uHwAhUAAAAAHQAAAAAQBw&utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAESABo9Ch8SGzoZSnVodSwgTXVtYmFpLCBNYWhhcmFzaHRyYRoAEhoSFAoHCOUPEAUYGRIHCOUPEAUYGhgBMgIQACoPCgsoAUoCIAE6A0lOUhoA"  # "https://www.travelguru.com/hotels/5-star-hotels-near-juhu-beach-in-mumbai"
#     r = requests.get(url)
#     htmlContent = r.content
#     soup = BeautifulSoup(htmlContent, 'html.parser')
#     # From the html data specific class data is been extracted usind find_all
#     price_data = soup.find_all('a', class_="OxGZuc W8vlAc lRagtb")
#     price_data = list(price_data)
#     rating_data = soup.find_all('a', class_="spNMC lRagtb")
#     rating_data = list(rating_data)
#     #String slicing is done extract required lines of data
#     ABCD = []
#     for a in price_data:
#         pattern = """
#         (?P<Price>[\d][\,][\d]+)
#         (\,\ )
#         (?P<Hotel>.*)
#         (\"\ class\=\"OxGZuc\ W8vlAc\ lRagtb\")
#         """
#         for item in re.finditer(pattern, str(a), re.VERBOSE):
#             ABCD.append(item.groupdict())
#     BCDE = []        
#     for a in rating_data:
#         pattern = """
#         (?P<Rating>[[\d+].*)
#         (\ out\ of\ 5\ stars\ from\ )
#         (?P<Total_Reviews>.*)
#         (\ reviews\,\ )
#         (?P<Hotels>.*)
#         (\"\ class\=\"spNMC\ lRagtb\")
    
#         """
#     for item in re.finditer(pattern, str(a), re.VERBOSE):
#             BCDE.append(item.groupdict())
#     CDEF = []
#     for i in range(0, len(ABCD)):
#         for j in range(0, len(BCDE)):
#             if  ABCD[i]['Hotel'] == BCDE[j]['Hotels']:
#                 CDEF.append(BCDE[j])
#         df1 = pd.DataFrame(ABCD)
#         df2 = pd.DataFrame(CDEF)
#         df = pd.concat([df1, df2], axis=1)
#     return df
# hotels_data()

In [25]:
# import time
# def calcProd():
#     product = 1
#     for i in range(1, 100000):
#         product = product * i
#     return product
# startTime = time.time()
# prod = calcProd()
# endTime = time.time()
# print('The result is %s digits long.' % (len(str(prod))))
# print('Took %s seconds to calculate.' %(endTime-startTime))

In [26]:
# import time
# for i in range(3):
#     print('Tick')
#     time.sleep(1)
#     print('Tock')
#     time.sleep(1)

In [27]:
# import pandas as pd
# import numpy as  np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import requests
# import re
# from bs4 import BeautifulSoup
# url = "https://www.google.com/travel/hotels/Mumbai?g2lb=2502548%2C4258168%2C4270442%2C4306835%2C4317915%2C4371335%2C4401769%2C4419364%2C4429192%2C4482438%2C4486153%2C4509341%2C4524134%2C4533547%2C4533548%2C4533882%2C4536454%2C4547627%2C4270859%2C4284970%2C4291517&hl=en-IN&gl=in&cs=1&ssta=1&ap=aAE&q=juhu%20hotels&rp=EMaW74Pl8vaAChCmiL67me393GwQqqT4zp6RhsMSEPOBovqAg7-zBzgBQABIAqIBGUp1aHUsIE11bWJhaSwgTWFoYXJhc2h0cmE&ictx=1&sa=X&ved=0CAAQ5JsGahcKEwjYj-Oj4uHwAhUAAAAAHQAAAAAQBw&utm_campaign=sharing&utm_medium=link&utm_source=htls&ts=CAESABo9Ch8SGzoZSnVodSwgTXVtYmFpLCBNYWhhcmFzaHRyYRoAEhoSFAoHCOUPEAUYGRIHCOUPEAUYGhgBMgIQACoPCgsoAUoCIAE6A0lOUhoA"  # "https://www.travelguru.com/hotels/5-star-hotels-near-juhu-beach-in-mumbai"
# r = requests.get(url)
# htmlContent = r.content
# soup = BeautifulSoup(htmlContent, 'html.parser')
# # From the html data specific class data is been extracted usind find_all
# price_data = soup.find_all('a', class_="OxGZuc W8vlAc lRagtb")
# price_data = list(price_data)
# rating_data = soup.find_all('a', class_="spNMC lRagtb")
# rating_data = list(rating_data)
# # String slicing is done extracting required lines of data
# ABCD = []
# for a in price_data:
#     pattern = """
#     (?P<Price>[\d].*[\,][\d]+)
#     (\,\ )
#     (?P<Hotel>.*)
#     (\"\ class\=\"OxGZuc\ W8vlAc\ lRagtb\")
#     """
#     for item in re.finditer(pattern, str(a), re.VERBOSE):
#         ABCD.append(item.groupdict())
# BCDE = []        
# for a in rating_data:
#     pattern = """
#     (?P<Rating>[\d+].*)
#     (\ out\ of\ 5\ stars\ from\ )
#     (?P<Review>.*)
#     (\ reviews\,\ )
#     (?P<Hotels>.*)
#     (\"\ class\=\"spNMC\ lRagtb\")
#     """
#     for item in re.finditer(pattern, str(a), re.VERBOSE):
#         BCDE.append(item.groupdict())
# CDEF = []
# for i in range(0, len(ABCD)):
#     for j in range(0, len(BCDE)):
#         if  ABCD[i]['Hotel'] == BCDE[j]['Hotels']:
#             CDEF.append(BCDE[j])
# df1 = pd.DataFrame(ABCD)
# df2 = pd.DataFrame(CDEF)
# # Merging two data frames
# df = pd.concat([df1, df2], axis=1)
# # Converting series data to numeric data
# df.Price = df.Price.str.replace(',','')
# df.Price = pd.to_numeric(df.Price)
# df.Rating = pd.to_numeric(df.Rating)
# df.Review = df.Review.str.replace(',','')
# df.Review = pd.to_numeric(df.Review)
# # Sort data
# df = df.sort_values('Price', ascending=False)
# df

In [28]:
# CDEF = []
# for i in range(0, len(ABCD)):
#     for j in range(0, len(CDEF)):
#         if  ABCD[i]['Hotel'] == BCDE[j]['Hotels']:
#             CDEF.append(BCDE[j])
# df1 = pd.DataFrame(ABCD)
# df2 = pd.DataFrame(CDEF)
# # Merging two data frames
# df = pd.concat([df1, df2], axis=1)
# df

In [29]:
import datetime
datetime.datetime.now()

datetime.datetime(2021, 6, 1, 2, 35, 23, 510315)

In [30]:
dt = datetime.datetime(2015, 10, 21, 16, 29, 0)
dt.year, dt.month, dt.day

(2015, 10, 21)

In [31]:
dt.hour, dt.minute, dt.second

(16, 29, 0)

In [32]:
datetime.datetime.fromtimestamp(1000000)

datetime.datetime(1970, 1, 12, 19, 16, 40)

In [33]:
datetime.datetime.fromtimestamp(time.time())

NameError: name 'time' is not defined

In [ ]:
delta = datetime.timedelta(days=11, hours=10, minutes=9, seconds=8)
delta.days, delta.seconds, delta.microseconds
delta.total_seconds()

In [ ]:
str(delta)

In [ ]:
# '''Multithreading
# To introduce the concept of multithreading, let’s look at an example situa-
# tion. Say you want to schedule some code to run after a delay or at a specific
# time. You could add code like the following at the start of your program:'''
# import time, datetime
# startTime = datetime.datetime(2029, 10, 31, 0, 0, 0)
# while datetime.datetime.now() < startTime:
#     time.sleep(1)
#     print('Program now starting on Halloween 2029')

In [ ]:
# import threading, time
# print('Start of program.')

# def takeANap():
#     time.sleep(5)
#     print('Wake up!')
# threadObj = threading.Thread(target=takeANap)
# threadObj.start()

# print('End of program.')

In [ ]:
import threading
threadObj = threading.Thread(target=print, args=['Cats', 'Dogs', 'Frogs'],kwargs={'sep': ' & '})
threadObj.start()

In [ ]:
threadObj = threading.Thread(target=print('Cats', 'Dogs', 'Frogs', sep=' & '))
threadObj.start()

In [ ]:
# import requests, os, bs4, threading
# os.makedirs('xkcd', exist_ok=True) # store comics in ./xkcd

# def downloadXkcd(startComic, endComic):
#     for urlNumber in range(startComic, endComic):
#         # Download the page.
#         print('Downloading page http://xkcd.com/%s...' % (urlNumber))
#         res = requests.get('http://xkcd.com/%s' % (urlNumber))
#         res.raise_for_status()

#         soup = bs4.BeautifulSoup(res.text)
#         # Find the URL of the comic image.

#         comicElem = soup.select('#comic img')
#         if comicElem == []:
#             print('Could not find comic image.')
#         else:
#             comicUrl = comicElem[0].get('src')
#             # Download the image.
#             print('Downloading image %s...' % (comicUrl))
#             res = requests.get(comicUrl)
#             res.raise_for_status()

#             # Save the image to ./xkcd.
#             imageFile = open(os.path.join('xkcd', os.path.basename(comicUrl)), 'wb')
#             for chunk in res.iter_content(100000):
#                 imageFile.write(chunk)
#                 imageFile.close()
# # TODO: Create and start the Thread objects.
# # TODO: Wait for all threads to end.

# # Create and start the Thread objects.
# downloadThreads = []              # a list of all the Thread objects
# for i in range(0, 1400, 100):     # loops 14 times, creates 14 threads
#     downloadThread = threading.Thread(target=downloadXkcd, args=(i, i + 99))
#     downloadThreads.append(downloadThread)
#     downloadThread.start()

# # Wait for all threads to end.
# for downloadThread in downloadThreads:
#     downloadThread.join()
# print('Done.')

In [ ]:
# Launching Other Programs from Python
# import subprocess
# subprocess.Popen('firefox')

In [ ]:
# import subprocess
# subprocess.Popen('libreoffice')

In [ ]:
# calcProc = subprocess.Popen('firefox')
# calcProc.poll() == None

In [ ]:
# import time , subprocess,os
# from subprocess import call

# def countDown():
#     for i in range (10):
#         print(i, end=' ')
#         time.sleep(1)

# countDown()
# subprocess.call(['xdg-open', '/home/shwetal/alarm.mp3'])

In [ ]:
#Importing all the necessary libraries to form the alarm clock:

# from tkinter import *
# import datetime
# import time, subprocess, os

# def alarm(set_alarm_timer):
#     while True:
#         time.sleep(1)
#         current_time = datetime.datetime.now()
#         now = current_time.strftime("%H:%M:%S")
#         date = current_time.strftime("%d/%m/%Y")
#         print("The Set Date is:",date)
#         print(now)
#         if now == set_alarm_timer:
#             print("Time to Wake up")
#         subprocess.call(['xdg-open', '/home/shwetal/alarm.mp3'])
#         break
# def actual_time():
#     set_alarm_timer = f"{hour.get()}:{min.get()}:{sec.get()}"
#     alarm(set_alarm_timer)
    
# clock = Tk()
# clock.title("Alarm Clock")
# clock.geometry("400x200")
# time_format=Label(clock, text= "Enter time in 24 hour format!", fg="red",bg="black",font="Arial").place(x=100,y=120)
# addTime = Label(clock,text = "Hour  Min   Second",font=60).place(x = 137)
# setYourAlarm = Label(clock,text = "When to wake you up",fg="blue",relief = "solid",font=("Helevetica",8,"bold")).place(x=1, y=29)
# # The Variables we require to set the alarm(initialization):
# hour = StringVar()
# min = StringVar()
# sec = StringVar()
# #Time required to set the alarm clock:
# hourTime= Entry(clock,textvariable = hour,bg = "pink",width = 15).place(x=137,y=26)
# minTime= Entry(clock,textvariable = min,bg = "pink",width = 15).place(x=180,y=26)
# secTime = Entry(clock,textvariable = sec,bg = "pink",width = 15).place(x=230,y=26)
# #To take the time input by user:
# submit = Button(clock,text = "Set Alarm",fg="red",width = 10,command = actual_time).place(x =150,y=70)
# clock.mainloop()
#Execution of the window

In [ ]:
# Sending Email
# import smtplib
# smtpObj = smtplib.SMTP('smtp.mail.yahoo.com', 587)
# smtpObj.ehlo()
# smtpObj.starttls()

# smtpObj.login('mail_id', 'password')

# smtpObj.sendmail('receiver_mail_id', 'receiver_mail_id', 
#                  'Subject: Solong.\nDear Shwetal, so long and thanks for all the stuff. Sincerely, Shwetal')

# smtpObj.quit()

# Connecting to an SMTP Server
# smtpObj = smtplib.SMTP('smtp.mail.yahoo.com', 587)
# type(smtpObj)

# smtpObj = smtplib.SMTP_SSL('smtp.mail.yahoo.com', 465)

# Sending the SMTP “Hello” Message
# smtpObj.ehlo()
                 
# Starting TLS Encryption
# smtpObj.starttls()
                 
# Logging in to the SMTP Server
# smtpObj.login('my_email_address@gmail.com', 'MY_SECRET_PASSWORD')
                 
# Sending an Email
# smtpObj.sendmail('my_email_address@gmail.com', 'recipient@example.com','Subject: So long.\nDear Alice, so long and thanks for all the fish. Sincerely,Bob')

# Disconnecting from the SMTP Server
# smtpObj.quit()

In [ ]:
# import email, smtplib, ssl
# import os
# from email import encoders
# from email.mime.base import MIMEBase
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText

# subject = "An email with attachment from Python"
# body = "This is an email with attachment sent from Python"
# sender_email = "python.hoteldata@gmail.com"
# receiver_email = "python.hoteldata@gmail.com"
# password = input("Type your password and press enter:")

# # Create a multipart message and set headers
# message = MIMEMultipart()
# message["From"] = sender_email
# message["To"] = receiver_email
# message["Subject"] = subject
# message["Bcc"] = receiver_email  # Recommended for mass emails

# # Add body to email
# message.attach(MIMEText(body, "plain"))

# filename = "hotel.pdf"  # In same directory as script

# # Open PDF file in binary mode
# with open(filename, "rb") as attachment:
#     # Add file as application/octet-stream
#     # Email client can usually download this automatically as attachment
#     part = MIMEBase("application", "octet-stream")
#     part.set_payload(attachment.read())

# # Encode file in ASCII characters to send by email    
# encoders.encode_base64(part)

# # Add header as key/value pair to attachment part
# part.add_header("Content-Disposition",f"attachment; filename= {filename}")

# # Add attachment to message and convert message to string
# message.attach(part)
# text = message.as_string()

# # Log in to server using secure context and send email
# context = ssl.create_default_context()
# with smtplib.SMTP("smtp.gmail.com", 587) as server:
#     server.ehlo()
#     server.starttls()
#     server.ehlo()
#     server.login(sender_email, password)
#     server.sendmail(sender_email, receiver_email, text)
#     server.quit()

In [ ]:
# Python program to explain os.environ object 
  
# importing os module 
# import os
# import pprint
  
# Get the list of user's
# environment variables
# env_var = os.environ
  
# Print the list of user's
# environment variables
# print("User's Environment variable:")
# pprint.pprint(dict(env_var), width = 1)

In [ ]:
# Python program to explain os.environ object 
  
# importing os module 
import os
  
# Get the value of
# 'HOME' environment variable
home = os.environ['HOME']
  
# Print the value of
# 'HOME' environment variable
print("HOME:", home)
  
# Get the value of
# 'JAVA_HOME' environment variable
# using get operation of dictionary
java_home = os.environ.get('HOME')
  
# Print the value of
# 'JAVA_HOME' environment variable
print("HOME:", java_home)

In [ ]:
# Python program to explain os.environ object 
  
# importing os module 
# import os

# Print the value of
# 'JAVA_HOME'  environment variable 
# print("HOME:", os.environ['HOME'])
  
# Modify the value of
# 'JAVA_HOME'  environment variable 
# os.environ['HOME'] = '/home/shwetal'
  
# Print the modified value of
# 'JAVA_HOME' environment variable
# print("Modified HOME:", os.environ['HOME'])

In [ ]:
# from datetime import datetime
# from threading import Timer

# x=datetime.today()
# y=x.replace(day=x.day+1, hour=1, minute=0, second=0, microsecond=0)
# delta_t=y-x

# secs=delta_t.seconds+1

# def hello_world():
#     print("hello world")
#     #...

# t = Timer(secs, hello_world)
# t.start()